In [3]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ['https://www.googleapis.com/auth/drive']

In [4]:
creds = None

if os.path.exists("credentials/token.json"):
    creds = Credentials.from_authorized_user_file("credentials/token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials/google_creds.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("credentials/token.json", "w") as token:
        token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=723729305832-sbh9mmpj0apcrea0gt1rifidfi3utkmq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54112%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=aPFsWq35XsdQAlunBn1irMcJgBWfq1&access_type=offline


In [5]:
import io
from googleapiclient.http import MediaIoBaseDownload

service = build("drive", "v3", credentials=creds)

query = f"'1kzq60ZAhyl9Z7uKe9T0bsr8NrOJKhuP9' in parents"
results = service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found in the folder.')
else:
    print('Files in folder:')
    for item in items:
        file_name = item['name']
        print(f"{item['name']} ({item['id']})")
        request = service.files().get_media(fileId=item['id'])
        file_path = os.path.join('downloads', file_name)  # Download files to the 'downloads' directory

        # Create the 'downloads' directory if it doesn't exist
        if not os.path.exists('downloads'):
            os.makedirs('downloads')

        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress for {file_name}: {int(status.progress() * 100)}%")

        # Save the file to the disk
        with open(file_path, 'wb') as f:
            f.write(fh.getvalue())
        print(f"Downloaded {file_name} to {file_path}")
        service.files().delete(fileId=item['id']).execute()

Files in folder:
model_gpu.pt (1wNYEDbFx4nqhV6JJ8AurgUlJjZxWrQ2X)
Download progress for model_gpu.pt: 100%
Downloaded model_gpu.pt to downloads\model_gpu.pt
model.pt (1CK-xWyWSlTHy7T5PlZz3pg-gliDQc7nW)
Download progress for model.pt: 100%
Downloaded model.pt to downloads\model.pt
untrained.pt (1dZEfcvB61u1gIJRTvnjFuJX9qqb4tBC_)
Download progress for untrained.pt: 100%
Downloaded untrained.pt to downloads\untrained.pt
